In [1]:
import os,pickle
os.chdir("/home/bruce1996/repo/LIHC_anomaly_detection/")
from utils.gsea import *
from utils.go2gene_dict import *
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
namespace = ['biological_process','cellular_component','molecular_function']
graph_d = dict()
for domain in namespace :
    with open("/home/bruce1996/data/GO/networkx/%s.go" % domain,'rb') as f :
        g = pickle.load(f)
    f.close()
    graph_d[domain] = g

In [3]:
biomart = pd.read_csv("/home/bruce1996/data/LIHC_anomaly_detection/data/coding_gene_info/biomart_protein_coding_gene.txt",sep='\t')
gene2go_matrix = create_gene2go_matrix(biomart=biomart,graph_dict=graph_d,threads=24)

In [3]:
import json
with open("/home/bruce1996/data/GO/go2namespace.json",'rb') as f :
    go2namespace = json.load(f)
f.close()

In [4]:
with open("/home/bruce1996/data/GO/networkx/go2gene_dict.pkl",'rb') as f :
        go2gene_dict = pickle.load(f)
f.close()

In [8]:
gene2go_matrix = remove_go_by_level(gene2go_matrix,go2namespace,graph_d)

In [6]:
experiment_evidence_code = ['EXP','IDA','IPI','IMP','IGI','IEP','HTP','HDA','HMP','HGI','HEP']
idx = [x in experiment_evidence_code for x in biomart['GO term evidence code']]
biomart = biomart.loc[idx,:]
idx = np.where((biomart['GO domain'] != np.NaN) & (biomart['GO term accession'].isnull() == False),True,False)
biomart = biomart.loc[idx,["Gene stable ID","GO term accession",'GO domain']]

In [7]:
go2gene_dict = create_go2gene_dict(gene2go_matrix,biomart)

### tmp

In [18]:
gsea_res = pd.read_csv("/home/bruce1996/data/LIHC_anomaly_detection/manifold_transformation/gsea_summary/1_1_against_1_2_activated_gsea_summary.txt",sep='\t',index_col=0)
gsea_res.drop(['GO_level.1'],axis=1,inplace=True)

In [29]:
gsea_res

,GO_level,#Nodes,namespace,Description,Pesudo-F,Origin_ES,ES_median,ES_std
GO:0001580,6,44,biological_process,detection of chemical stimulus involved in sen...,0.000,0.529368,0.128674,0.073438
GO:0014910,6,3,biological_process,regulation of smooth muscle cell migration,0.000,0.939398,0.295422,0.179370
GO:0062188,6,3,molecular_function,"anandamide 11,12 epoxidase activity",0.000,0.977172,0.369197,0.221317
GO:0062189,6,4,molecular_function,"anandamide 14,15 epoxidase activity",0.000,0.954103,0.285381,0.182050
GO:0005184,6,9,molecular_function,neuropeptide hormone activity,0.000,0.600249,0.191017,0.107936
...,...,...,...,...,...,...,...,...
GO:0001654,5,20,biological_process,eye development,0.049,0.266056,0.123960,0.071439
GO:0004445,8,1,molecular_function,inositol-polyphosphate 5-phosphatase activity,0.049,0.953694,0.539636,0.289330
GO:0051612,6,1,biological_process,negative regulation of serotonin uptake,0.049,0.941421,0.527718,0.284357
GO:0008700,5,1,molecular_function,4-hydroxy-2-oxoglutarate aldolase activity,0.049,0.951559,0.471807,0.292613
